In [16]:
import requests
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, datetime, timedelta, timezone
from scipy.cluster.hierarchy import linkage, dendrogram
from time import sleep
from random import randint

# # API Camp to Camp - 

In [2]:
# #Export data from C2C API

url = 'https://api.camptocamp.org/outings'
france = '&a=14274'
req_activities = {'?act=skitouring':310,'?act=rock_climbing':330,'?act=mountain_climbing':100,'?act=snow_ice_mixed':100,'?act=ice_climbing':100,'?act=hiking':100}

request_num = 0
result=[]

for k,v in req_activities.items():
    for i in range(0,v):
        if i == 0 : 
            api_path = url + k + france
        else:
            api_path = url + k + france + '&offset=' + str(i*30)            
        print(api_path)
        r = requests.get(api_path)
        result.append(r.json())
        request_num += 1
        sleep(randint(1,3))
        
print("Nombre total de reqûetes :" + str(request_num))
        
path = 'C:/Users/Mossuz Damien/Documents/Python Scripts/01 - C2C API/input_/'

with open(path + "merged_file.json", "w",encoding='utf-8') as outfile:
     json.dump(result, outfile, ensure_ascii=False,indent=4)

#Import merged results -        
with open(path + "merged_file.json", "r",encoding='utf-8') as infile:
     json_data = json.load(infile)

#Check - 
print(len(json_data))
print(json_data)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Mossuz Damien/Documents/Python Scripts/01 - C2C API/input_/merged_file.json'

# # Parse json file & data cleaning

In [ ]:
#>>>Y-a-t-il un moyen de traiter plus rapidement les imports que je récupère?
#>>>Créer des fonctions pour ceux que je n'arrive pas à récupérer avec json_normalize - 

#Step 1: flatten json files - iteratively -

d = pd.DataFrame()

# d= json_normalize(json_data[0]['documents'],max_level=1)

for k in range(len(json_data)):
    #Voir comment faire en partant d'un fichier JSON enregistré sur mon PC

    #Normalize json
    d=d.append(json_normalize(json_data[k]['documents'],max_level=1))
    # d= json_normalize(json_data,max_level=1)


    #Geometry - Voir si je peux simplifier?
    # arr_lat=[]
    # arr_long=[]
    # arr_id=[]

    # for k in range(len(json_data['documents'])):
    #     arr_lat.append(json.loads(json_data['documents'][k]['geometry']['geom'])['coordinates'][0])
    #     arr_long.append(json.loads(json_data['documents'][k]['geometry']['geom'])['coordinates'][1])
    #     arr_id.append(json_data['documents'][k]['document_id'])

    # d['latitude']=arr_lat
    # d['longitude']=arr_long
    # d['doc_id_test']=arr_id

    #Check - OK
    # print(np.sum(d['document_id']==d['doc_id_test']))

    #Locales - OK - A adapter pour la boucle
    # d_locales = json_normalize(json_data['documents'], record_path=['locales'], meta='document_id')
    # d = pd.merge(left=d, right=d_locales,on='document_id')

    #Areas - A d'utilité -> on ne prend pas
    # test = json_normalize(json_data['documents'], record_path=['areas'])
    # test
    # test.loc[0,'locales']

    # print(d.columns)

# for k in d.columns:
#     print(str(k) + ' - ' + str(d.loc[3,k]))

d.head()
# print(d.shape)

In [ ]:
#COLUMNS Selection & order - ID, user, acitivité, date, difficulté, condition - 
cols = ['document_id','activities','date_start','date_end','condition_rating','author.name','author.user_id','labande_global_rating','ski_rating','global_rating','engagement_rating','hiking_rating','equipment_rating','rock_free_rating','ice_rating','elevation_max','height_diff_up','height_diff_difficulties']
d=d[cols]

#Reset index
d.reset_index(inplace=True, drop=True)


In [ ]:
#Cleaning Activities - 
def binary(x,a):
    if a in x:
        return True
    else:
        return False

activities=['skitouring', 'rock_climbing','mountain_climbing','ice_climbing','hiking','snow_ice_mixed']

for activity in activities:
    d[activity]=d.activities.apply(lambda x: binary(x,activity))

d.drop('activities', inplace=True, axis=1)

#Drop_duplicates - Doublon document_id à cause du multi activités
print('Nombre de doublons de document_id : {}'.format(np.sum(d.duplicated(subset='document_id'))))
d.drop_duplicates(inplace=True)
print('Nombre de doublons de document_id : {}'.format(np.sum(d.duplicated(subset='document_id'))))

#Check - Date min pour chaque activité
#>>>>>Code à revoir
# d_date_min = d.groupby(by='activities')['date_start'].min()
# print(d_date_min)

In [ ]:
#- DATA Types -

# Categorical data -
col_to_category=['condition_rating','labande_global_rating','ski_rating','global_rating','engagement_rating','hiking_rating','equipment_rating','rock_free_rating','ice_rating']

for c in col_to_category:
    d[c]=d[c].astype('category')
    
#>>>Category avec ordre pour toutes les cotations    
d['global_rating'] = pd.Categorical(d['global_rating'], ['F','F+','PD-','PD','PD+','AD-','AD','AD+','D-','D','D+','TD-','TD','TD+','ED-','ED','ED+','ED4','ED5','ED7'],ordered=True)
d['global_rating_codes'] = d['global_rating'].cat.codes
d['rock_free_rating'] = pd.Categorical(d['rock_free_rating'], ['2','3a','3b','3c','4a','4b','4c','5a','5a+','5b','5b+','5c','5c+','6a','6a+','6b','6b+','6c','6c+','7a','7a+','7b','7b+','7c','7c+','8a','8a+'],ordered=True)
d['rock_free_rating_codes'] = d['rock_free_rating'].cat.codes
#X[categorical_cols] = X[categorical_cols].apply(lambda col: le.fit_transform(col))

# Datetime data
col_to_date=['date_end','date_start']
d[col_to_date]=d[col_to_date].apply(lambda x: pd.to_datetime(x))

# Float
col_to_float =['elevation_max','height_diff_up']
d[col_to_float]=d[col_to_float].astype(float)

# Int
col_to_int=['document_id','author.user_id']
d[col_to_int]=d[col_to_int].astype(int)

# CHECK
print(d.info())
d.head()

d['month']=d.date_start.dt.month
